
这里以中文BERT为例，实现提及聚类：

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")

# 进行分词
sentence="小明给小红一束花，她很高兴。"
subtokenized_sentence=tokenizer.tokenize(sentence)
subtokenized_sentence = [tokenizer._cls_token] + \
    subtokenized_sentence + [tokenizer._sep_token]
subtoken_ids_sentence = tokenizer.convert_tokens_to_ids(\
    subtokenized_sentence)
print(subtokenized_sentence)
print(subtoken_ids_sentence)

# 计算对应的特征
outputs = model(torch.Tensor(subtoken_ids_sentence).\
    unsqueeze(0).long())
hidden_states = outputs[0]
print(hidden_states.shape)

['[CLS]', '小', '明', '给', '小', '红', '一', '束', '花', '，', '她', '很', '高', '兴', '。', '[SEP]']
[101, 2207, 3209, 5314, 2207, 5273, 671, 3338, 5709, 8024, 1961, 2523, 7770, 1069, 511, 102]
torch.Size([1, 16, 768])


假设已经通过提及检测模型找到了句子中的提及，这里用每个提及的第一个子词（在中文中也就是第一个字）作为词特征：

In [2]:
# 提及的跨度，假设(-1,0)表示[CLS]的跨度，用于表示空提及[NA]，
# 在实际训练中也可以额外定义个空提及符号
mention_spans = [(-1,0),(0,2),(3,5),(10,11)]
word_features = torch.concat([hidden_states[0,x+1].unsqueeze(0)\
    for (x,y) in mention_spans],0)
print(word_features.shape)

torch.Size([4, 768])


首先，通过双仿射函数计算打分。

In [3]:
import sys
sys.path.append('../code')
from utils import Biaffine
biaffine = Biaffine(word_features.shape[1])

# 对word_features进行打分
scores = biaffine(word_features.unsqueeze(0),\
    word_features.unsqueeze(0))
# 由于只关注当前提及之前的提及是否与其进行共指，
# 因此将它转换为下三角函数，并且为上三角部分置为负无穷：
scores = scores.tril(diagonal=-1)
inf_mask = torch.zeros_like(scores)-torch.inf
inf_mask = inf_mask.triu()
scores += inf_mask
print(scores)
print(scores.argmax(-1)[:,1:])

tensor([[[      -inf,       -inf,       -inf,       -inf],
         [   58.9533,       -inf,       -inf,       -inf],
         [  571.2849,  -515.9794,       -inf,       -inf],
         [ -341.3851,  -697.8577, -1196.0930,       -inf]]],
       grad_fn=<AddBackward0>)
tensor([[0, 0, 0]])


由于模型未经过训练，因此仅通过双仿射函数初始化获得结构显然是错误的。我们可以训练模型，计算损失函数计算方式如下：

In [4]:
# 只计算除了[NA]以外的提及的损失
target = torch.Tensor([0,0,1]).long()
loss_func = torch.nn.NLLLoss()
loss = loss_func(torch.nn.functional.log_softmax(scores[:,1:].\
    squeeze(0),-1),target)
print(loss)

tensor(118.8242, grad_fn=<NllLossBackward0>)


接下来通过点积计算打分。

In [5]:
scores2 = torch.matmul(word_features,word_features.T)
scores2 = scores2.tril(diagonal=-1)
inf_mask = torch.zeros_like(scores2)-torch.inf
inf_mask = inf_mask.triu()
scores2 += inf_mask
print(scores2)
print(scores2.argmax(-1)[1:])

tensor([[    -inf,     -inf,     -inf,     -inf],
        [235.2013,     -inf,     -inf,     -inf],
        [188.3145, 267.1166,     -inf,     -inf],
        [221.3709, 101.3911, 292.7802,     -inf]], grad_fn=<AddBackward0>)
tensor([0, 1, 2])
